In [1]:
from utils.load_results import *
from utils.plot_helpers import *

import pandas as pd
#import seaborn as sns
from matplotlib import pyplot as plt
plt.style.use('default')
import torch
from utils.analysis_from_interaction import *
from language_analysis_local import TopographicSimilarityConceptLevel, encode_target_concepts_for_topsim
from utils.analysis_tools_lazimpa import *
import os
if not os.path.exists('analysis'):
    os.makedirs('analysis')
#import plotly.express as px

In [2]:
datasets = ('(3,4)', '(3,8)', '(4,4)')
n_attributes = (3, 3, 4)
n_values = (4, 8, 4)
n_epochs = 300
n_datasets = len(datasets)
paths = ['results/' + d + '_game_size_10_vsf_3/' for d in datasets]

In [3]:
datasets = ['(4,4)',]
n_values = [4,]
n_attributes = [4,]
paths = ['results/' + d + '_game_size_10_vsf_3/' for d in datasets]

In [4]:
datasets = ['(3,4)',]
n_values = [4,]
n_attributes = [3,]
n_epochs = 300
paths = ['results/' + d + '_game_size_10_vsf_3/' for d in datasets]

In [3]:
context_unaware = False # whether original or context_unaware simulations are evaluated
lazy = True # whether the lazy agent is used
impatient = True
setting = ""
if not lazy:
    start = 'impatience_' if impatient else ''
    if context_unaware:
        setting = start + 'context_unaware'
    else:
        setting = start + 'context_aware'
        if impatient == False:
            setting = 'standard'
else:
    start = 'lazimpa' if impatient else 'lazy'
    if context_unaware:
        setting = start + '_context_unaware'
    else:
        setting = start + '_context_aware'
setting

'lazimpa_context_aware'

### Determine vocab size

In [4]:
from collections import Counter

In [5]:
# go through all datasets
for i, d in enumerate(datasets):
    
    for run in range(5):
        # select first run
        interaction = load_interaction(paths[i],setting,run,n_epochs)
        
        messages = retrieve_messages(interaction)
        all_symbols = [symbol for message in messages for symbol in message]
        symbol_counts = Counter(all_symbols)
        print("Run ",run)
        print(symbol_counts)
        print(len(symbol_counts))

results/(3,4)_game_size_10_vsf_3//lazimpa_context_aware/0/interactions/train/epoch_300/interaction_gpu0
Run  0
Counter({0: 4167, 2: 352, 12: 312, 13: 274, 3: 264, 14: 219, 4: 215, 1: 207, 10: 196, 7: 181, 9: 149, 8: 138, 6: 118, 11: 113, 15: 96, 5: 47})
16
results/(3,4)_game_size_10_vsf_3//lazimpa_context_aware/1/interactions/train/epoch_300/interaction_gpu0
Run  1
Counter({0: 4031, 8: 449, 7: 345, 1: 337, 6: 305, 5: 224, 9: 224, 10: 170, 11: 166, 13: 165, 14: 139, 3: 131, 12: 124, 4: 120, 2: 103, 15: 103})
16
results/(3,4)_game_size_10_vsf_3//lazimpa_context_aware/2/interactions/train/epoch_300/interaction_gpu0
Run  2
Counter({0: 4043, 4: 435, 9: 347, 15: 287, 2: 284, 3: 229, 1: 211, 14: 210, 12: 208, 5: 160, 7: 158, 10: 153, 6: 127, 11: 126, 13: 116, 8: 102})
16
results/(3,4)_game_size_10_vsf_3//lazimpa_context_aware/3/interactions/train/epoch_300/interaction_gpu0
Run  3
Counter({0: 4314, 12: 335, 8: 332, 11: 258, 1: 251, 14: 247, 10: 239, 5: 229, 15: 169, 3: 155, 7: 150, 9: 127, 13:

### See which messages are produced for which concepts

In [10]:
datasets = ['(3,4)']
n_values = [4]
n_attributes = [3]
n_epochs = 300
n_datasets = len(datasets)
paths = ['results/' + d + '_game_size_10_vsf_3' for d in datasets]

In [9]:
# go through all datasets
run = 0
for i, d in enumerate(datasets):
    # select run
    interaction = load_interaction(paths[i],setting,run,n_epochs)
    messages = retrieve_messages(interaction,remove_after_eos=True)
    concepts, context_conds = retrieve_concepts_context(interaction,n_values[i])

    # get random qualitative samples
    #fixed_index = random.randint(0, n_attributes[i]-1) # define a fixed index for the concept
    n_fixed = random.randint(1, n_attributes[i]) # how many fixed attributes?
    # n_fixed = 3
    fixed_indices = random.sample(range(0, n_attributes[i]), k=n_fixed) # select which attributes are fixed
    #fixed_indices = [0, 1, 2]
    #fixed_values = random.randint(0, n_values[i]-1) # define a fixed value for this index
    fixed_values = random.choices(range(0, n_values[i]), k=n_fixed)
    #fixed_values = [0, 0, 0]
    print(n_fixed, fixed_indices, fixed_values)
    #index_threshold = 20000 # optional: define some index threshold to make sure that examples are not taken from the beginning of training
    # TODO: adapt this loop such that multiple indices can be fixed
    all_for_this_concept = []
    for idx, (t_objects, t_fixed) in enumerate(concepts):
        #if sum(t_fixed) == 1 and t_fixed[fixed_index] == 1:# and idx > index_threshold:
        if sum(t_fixed) == n_fixed and all(t_fixed[fixed_index] == 1 for fixed_index in fixed_indices):
            for t_object in t_objects:
                if all(t_object[fixed_index] == fixed_values[j] for j, fixed_index in enumerate(fixed_indices)):
                    all_for_this_concept.append((idx, t_object, t_fixed, context_conds[idx], messages[idx]))
                    fixed = t_fixed
    if len(all_for_this_concept) > 0:
        #sample = random.sample(all_for_this_concept, 20)
        sample = all_for_this_concept
        column_names = ['game_nr', 'object', 'fixed indices', 'context condition', 'message']
        df = pd.DataFrame(sample, columns=column_names)
        path_to_save = 'analysis/quali_' + str(d) + '_' + str(setting) + '_' + str(sample[0][1]) + ',' + str(fixed) + 'all.csv'
        print(path_to_save)
        df.to_csv(path_to_save, index=False)
        print('saved ' + 'analysis/quali_' + str(d) + '_' + str(setting) + '_' + str(sample[0][1]) + ',' + str(fixed) + 'all.csv')
    else:
        raise ValueError("sample for dataset " + str(d) + " could not be generated")

results/(3,4)_game_size_10_vsf_3//lazimpa_context_aware/0/interactions/train/epoch_300/interaction_gpu0
torch.Size([1762, 20, 12])
1 [1] [1]
analysis/quali_(3,4)_lazimpa_context_aware_[3. 1. 2.],[0. 1. 0.]all.csv
saved analysis/quali_(3,4)_lazimpa_context_aware_[3. 1. 2.],[0. 1. 0.]all.csv
results/(3,8)_game_size_10_vsf_3//lazimpa_context_aware/0/interactions/train/epoch_300/interaction_gpu0
torch.Size([11759, 20, 24])
1 [0] [4]
analysis/quali_(3,8)_lazimpa_context_aware_[4. 2. 3.],[1. 0. 0.]all.csv
saved analysis/quali_(3,8)_lazimpa_context_aware_[4. 2. 3.],[1. 0. 0.]all.csv
results/(4,4)_game_size_10_vsf_3//lazimpa_context_aware/0/interactions/train/epoch_300/interaction_gpu0
torch.Size([11939, 20, 16])
4 [2, 0, 3, 1] [0, 1, 2, 1]
analysis/quali_(4,4)_lazimpa_context_aware_[1. 1. 0. 2.],[1. 1. 1. 1.]all.csv
saved analysis/quali_(4,4)_lazimpa_context_aware_[1. 1. 0. 2.],[1. 1. 1. 1.]all.csv


In [10]:
# Groupby und Aggregation for analysis using pandas
def aggregate_messages(messages):
    message_counts = messages.value_counts().to_dict()
    unique_messages = list(message_counts.keys())
    frequencies = list(message_counts.values())
    return unique_messages, frequencies

def group_by_messages(df):
    grouped = df.groupby(['context condition','object','fixed indices'])['message'].apply(aggregate_messages).reset_index()
    grouped[['unique_messages', 'frequencies']] = pd.DataFrame(grouped['message'].tolist(), index=grouped.index)
    return grouped.drop('message', axis=1)

In [18]:
df = pd.read_csv('analysis/quali_(3,4)_lazimpa_context_aware_[3. 1. 2.],[0. 1. 0.]all.csv')
group_by_messages(df) # IMPORTANT: These two messages each are the same because of eos!!!

,context condition,object,fixed indices,unique_messages,frequencies
0,0,[0. 1. 0.],[0. 1. 0.],"[[10, 10, 0, 0]]",[4]
1,0,[0. 1. 1.],[0. 1. 0.],"[[10, 10, 0, 0], [10, 14, 0, 0]]","[3, 1]"
2,0,[0. 1. 2.],[0. 1. 0.],"[[10, 10, 0, 0], [10, 14, 0, 0]]","[3, 1]"
3,0,[0. 1. 3.],[0. 1. 0.],"[[10, 10, 0, 0], [10, 14, 0, 0]]","[1, 1]"
4,0,[1. 1. 0.],[0. 1. 0.],"[[10, 10, 0, 0], [10, 14, 0, 0]]","[4, 1]"
5,0,[1. 1. 1.],[0. 1. 0.],"[[10, 10, 0, 0], [10, 14, 0, 0]]","[5, 1]"
6,0,[1. 1. 2.],[0. 1. 0.],"[[10, 10, 0, 0]]",[5]
7,0,[1. 1. 3.],[0. 1. 0.],"[[10, 10, 0, 0]]",[4]
8,0,[2. 1. 0.],[0. 1. 0.],"[[10, 10, 0, 0], [10, 14, 0, 0]]","[3, 1]"
9,0,[2. 1. 1.],[0. 1. 0.],"[[10, 10, 0, 0], [10, 14, 0, 0]]","[2, 1]"


In [15]:
df = pd.read_csv('analysis/quali_(3,8)_lazimpa_context_aware_[4. 2. 3.],[1. 0. 0.]all.csv')
group_by_messages(df)

,context condition,object,fixed indices,unique_messages,frequencies
0,0,[4. 0. 0.],[1. 0. 0.],"[[17, 24, 0, 0]]",[1]
1,0,[4. 0. 1.],[1. 0. 0.],"[[17, 24, 0, 0]]",[2]
2,0,[4. 0. 3.],[1. 0. 0.],"[[17, 24, 0, 0]]",[2]
3,0,[4. 0. 4.],[1. 0. 0.],"[[17, 24, 0, 0]]",[2]
4,0,[4. 0. 5.],[1. 0. 0.],"[[17, 24, 0, 0]]",[2]
5,0,[4. 0. 7.],[1. 0. 0.],"[[17, 24, 0, 0]]",[4]
6,0,[4. 1. 2.],[1. 0. 0.],"[[17, 24, 0, 0]]",[1]
7,0,[4. 1. 3.],[1. 0. 0.],"[[17, 24, 0, 0]]",[1]
8,0,[4. 1. 6.],[1. 0. 0.],"[[17, 24, 0, 0]]",[2]
9,0,[4. 1. 7.],[1. 0. 0.],"[[17, 24, 0, 0]]",[1]


In [24]:
df = pd.read_csv('analysis/quali_(4,4)_lazimpa_context_aware_[1. 1. 0. 2.],[1. 1. 1. 1.]all.csv')
group_by_messages(df)

,context condition,object,fixed indices,unique_messages,frequencies
0,0,[1. 1. 0. 2.],[1. 1. 1. 1.],"[[3, 0, 0, 0, 0], [13, 0, 0, 0, 0], [9, 0, 0, ...","[40, 20, 10]"
1,1,[1. 1. 0. 2.],[1. 1. 1. 1.],"[[3, 0, 0, 0, 0], [13, 0, 0, 0, 0], [9, 0, 0, ...","[40, 10, 10]"
2,2,[1. 1. 0. 2.],[1. 1. 1. 1.],"[[9, 0, 0, 0, 0], [3, 0, 0, 0, 0], [9, 3, 0, 0...","[30, 10, 10, 10]"
3,3,[1. 1. 0. 2.],[1. 1. 1. 1.],"[[9, 15, 0, 0, 0], [13, 9, 0, 0, 0], [9, 3, 0,...","[30, 10, 10, 10]"


### Case study: concepts on different levels of abstraction

In [19]:
datasets = ['(3,4)']
n_values = [4]
n_attributes = [3]
n_epochs = 300
n_datasets = len(datasets)
paths = ['results/' + d + '_game_size_10_vsf_3' for d in datasets]

In [20]:
interaction = load_interaction(paths[0],setting,0,n_epochs)
messages = retrieve_messages(interaction)
concepts, context_conds = retrieve_concepts_context(interaction,n_values[0])

results/(3,4)_game_size_10_vsf_3/lazimpa_context_aware/0/interactions/train/epoch_300/interaction_gpu0
torch.Size([1762, 20, 12])


In [21]:
concept = np.array([0., 2., 0.,]) # Warum hat das Konzept 0 in der mitte fix
fixed = np.array([0., 1., 0.])
n_fixed = 1
fixed_indices = [1,]
fixed_values = [0, 1, 2]
print(n_fixed, fixed_indices, fixed_values)
#index_threshold = 20000 # optional: define some index threshold to make sure that examples are not taken from the beginning of training
# TODO: adapt this loop such that multiple indices can be fixed
all_for_this_concept = []
for idx, (t_objects, t_fixed) in enumerate(concepts):
    #if sum(t_fixed) == 1 and t_fixed[fixed_index] == 1:# and idx > index_threshold:
    if sum(t_fixed) == n_fixed and all(t_fixed[fixed_index] == 1 for fixed_index in fixed_indices):
        for t_object in t_objects:
            if all(t_object[fixed_index] == fixed_values[j] for j, fixed_index in enumerate(fixed_indices)):
                all_for_this_concept.append((idx, t_object, t_fixed, context_conds[idx], messages[idx]))
if len(all_for_this_concept) > 0:
    sample = random.sample(all_for_this_concept, 20)
    column_names = ['game_nr', 'object', 'fixed indices', 'context condition', 'message']
    df = pd.DataFrame(sample, columns=column_names)
    df.to_csv('analysis/quali_' + str(datasets[0]) + '_' + str(setting) + '_' + str(concept) + ',' + str(fixed) + '.csv', index=False)
    print('saved ' + 'analysis/quali_' + str(datasets[0]) + '_' + str(setting) + '_' + str(concept) + ',' + str(fixed) + '.csv')
else:
    raise ValueError("sample for dataset " + str(datasets[0]) + " could not be generated")

1 [1] [0, 1, 2]
saved analysis/quali_(3,4)_lazimpa_context_aware_[0. 2. 0.],[0. 1. 0.].csv


In [22]:
df = pd.read_csv('analysis/quali_(3,4)_lazimpa_context_aware_[0. 2. 0.],[0. 1. 0.].csv')
df

,game_nr,object,fixed indices,context condition,message
0,1477,[2. 0. 3.],[0. 1. 0.],0,"[7, 7, 0, 0]"
1,1208,[3. 0. 3.],[0. 1. 0.],0,"[7, 7, 0, 0]"
2,1021,[1. 0. 2.],[0. 1. 0.],0,"[7, 7, 0, 0]"
3,750,[2. 0. 3.],[0. 1. 0.],0,"[7, 7, 7, 0]"
4,980,[1. 0. 2.],[0. 1. 0.],0,"[7, 7, 0, 0]"
5,1477,[3. 0. 2.],[0. 1. 0.],0,"[7, 7, 0, 0]"
6,750,[0. 0. 0.],[0. 1. 0.],0,"[7, 7, 7, 0]"
7,980,[2. 0. 0.],[0. 1. 0.],0,"[7, 7, 0, 0]"
8,980,[3. 0. 0.],[0. 1. 0.],0,"[7, 7, 0, 0]"
9,1477,[1. 0. 3.],[0. 1. 0.],0,"[7, 7, 0, 0]"


### Dataset (3,4)